# `flair`: la librería NLP de Zalando Research

La compañia Zalando tiene necesidades de aplicar NLP en distintos ámbitos y su equipo de investigaación ha liberado recientemente [`flair`](https://github.com/zalandoresearch/flair), su librería de NLP.

`flair` permite acceder a funcionalidades muy interesantes para procesar lenguaje natural, algunas de ellas muy modernas como:

- [etiquetar morfo-sintácticamente](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md)
- extraer entidades
- clasificar automáticamente texto
- entrenar tus propios modelos para [construir otros clasificadores](https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f)
- [cargar vectores de palabras en decenas de lenguas](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md)
- [usar vectores contextuales como BERT, ELMo](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_4_ELMO_BERT_FLAIR_EMBEDDING.md)

Veamos cómo podemos acceder a algunas de sus funcionalidades.

## Análisis morfo-sintáctico

Para analizar sintácticamente un texto, necesitamos cargar un etiquetador con un modelo concreto de información morfo-sintáctica. Por ejemplo, uno capaz de analizar varias lenguas.


In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el analizador multi-idioma
tagger = SequenceTagger.load("pos-multi-fast")

2019-04-13 10:59:58,733 loading file /home/victor/.flair/models/pos-multi-fast.pt


In [3]:
sentence1 = Sentence("Facebook nació hace década y media tras una noche de copas de Mark Zuckerberg. ")
tagger.predict(sentence1)
# imprimimos el análisis
print(sentence1.to_tagged_string())

sentence2 = Sentence("Grand débat national: suivez Emmanuel Macron en direct de Bordeaux. ")
tagger.predict(sentence2)
# imprimimos el análisis
print(sentence2.to_tagged_string())

sentence3 = Sentence("Hier an der Zufahrt zur Startrampe 39A, wo vor 50 Jahren die gigantischen Saturn-Raketen der Apollo-Mondmissionen im Schneckentempo vorbeigefahren sind, prangen nun die blauen Lettern des Raumfahrtunternehmens von Elon Musk an einem Hangar.")
tagger.predict(sentence3)
# imprimimos el análisis
print(sentence3.to_tagged_string())

Facebook <PROPN> nació <VERB> hace <VERB> década <NOUN> y <CCONJ> media <NOUN> tras <ADP> una <DET> noche <NOUN> de <ADP> copas <NOUN> de <ADP> Mark <PROPN> Zuckerberg. <PROPN>
Grand <ADJ> débat <NOUN> national: <ADJ> suivez <VERB> Emmanuel <PROPN> Macron <PROPN> en <ADP> direct <NOUN> de <ADP> Bordeaux. <PROPN>
Hier <ADV> an <ADP> der <DET> Zufahrt <NOUN> zur <ADP> Startrampe <NOUN> 39A, <PROPN> wo <ADV> vor <ADP> 50 <NUM> Jahren <NOUN> die <DET> gigantischen <ADJ> Saturn-Raketen <NOUN> der <DET> Apollo-Mondmissionen <NOUN> im <ADP> Schneckentempo <NOUN> vorbeigefahren <VERB> sind, <AUX> prangen <VERB> nun <ADV> die <DET> blauen <ADJ> Lettern <NOUN> des <DET> Raumfahrtunternehmens <NOUN> von <ADP> Elon <PROPN> Musk <PROPN> an <ADP> einem <DET> Hangar. <NOUN>


## Reconocimiento de entidades

Para el reconocimiento de entidades varios modelos en diferentes lenguas. Aquí probamos con uno entrenado solo para inglés.

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el reconocedor de entidades
tagger = SequenceTagger.load("ner-fast")

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
2019-04-13 13:02:27,891 loading file /home/victor/.flair/models/en-ner-fast-conll03-v0.2.pt


In [3]:
# analizamos una oración
sentence = Sentence("Behind closed doors, freshman Rep. Alexandria Ocasio-Cortez threatened to put those voting with Republicans “on a list” for a primary challenge in the 2020 election.")
tagger.predict(sentence)

# imprimimos el análisis
print(sentence.to_tagged_string())

# iteramos por la entidades
for entity in sentence.get_spans("ner"):
    print(entity)
    
# o imprimimos la estructura de datos con el análisis completo
print(sentence.to_dict(tag_type="ner"))

Behind closed doors, freshman Rep. Alexandria <B-PER> Ocasio-Cortez <E-PER> threatened to put those voting with Republicans <S-MISC> “on a list” for a primary challenge in the 2020 election.
PER-span [6,7]: "Alexandria Ocasio-Cortez"
MISC-span [14]: "Republicans"
{'text': 'Behind closed doors, freshman Rep. Alexandria Ocasio-Cortez threatened to put those voting with Republicans “on a list” for a primary challenge in the 2020 election.', 'labels': [], 'entities': [{'text': 'Alexandria Ocasio-Cortez', 'start_pos': 35, 'end_pos': 59, 'type': 'PER', 'confidence': 0.9429651200771332}, {'text': 'Republicans', 'start_pos': 96, 'end_pos': 107, 'type': 'MISC', 'confidence': 0.999947190284729}]}


In [4]:
entity.tag

'MISC'

## Análisis de Opinión

También podemos utilizar un clasificador de textos y cargar el modelo entrenado con sentimiento, para poder detectar opiniones positivas y negativas.

In [5]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load("en-sentiment")

2019-04-13 13:05:56,199 loading file /home/victor/.flair/models/imdb.pt


/home/victor/.virtualenvs/ds/lib/python3.6/site-packages/torch/serialization.py:542: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


In [6]:
sentence1 = Sentence("I love ice-cream!")
classifier.predict(sentence1)
print("La frase '{}' es {}".format(sentence1.to_plain_string(), sentence1.labels))

sentence2 = Sentence("Don't ever go to this restaurant. The food was horrible :-(")
classifier.predict(sentence2)
print("La frase '{}' es {}".format(sentence2.to_plain_string(), sentence2.labels))

La frase 'I love ice-cream!' es [POSITIVE (1.0)]
La frase 'Don't ever go to this restaurant. The food was horrible :-(' es [NEGATIVE (1.0)]
